# Modeling

### Getting encoded data

Since we have to do feature selection again we have to do one hot encoding again

In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tflearn

C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\ten

Instructions for updating:
non-resource variables are not supported in the long term


C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dokee\anaconda3\envs\TensorEnv\

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
cleaned_df = pd.read_csv("cleaned_df.csv") 

In [3]:
cleaned_df = cleaned_df.drop(["COVID_HR1","COVID_HR2","IDEOLOGY4"],axis=1)
missing = ["HR11","IDEOLOGY7"]
for i in missing:
    mode = cleaned_df[i].mode()
    print(mode[0])
    fixed = []
    for value in cleaned_df[i]:
        if value == 'NA' or value != value:
            fixed.append(mode[0])
        else:
            fixed.append(value)
    cleaned_df[i] = fixed

B
C


In [4]:
for (columnName, columnData) in cleaned_df.iteritems(): 
    missingCount = (columnData == 'NA').sum() 
    missingCount += columnData.isnull().sum() 
    percentage = (missingCount/len(cleaned_df))*100 
    #print(percentage)
    if missingCount > 0:        
        print("Missing Values in column : ", columnName, ", missing rows: ", 
            missingCount, ", missing percentage: ",  percentage, '%') 
        if percentage > 50.0:
            too_many_missing_value_columns.append(columnName) 

### Feature selection
Feature we are going to use here is the ordinal IDEOLOGY5. IDEOLOGY5 is "If you had to describe your ideology along a Left-Right political spectrum, which of the following would come closest to how you would describe yourself?"

In [5]:
cleaned_df["IDEOLOGY5"].unique()

array([ 2,  0, -2, -1,  1], dtype=int64)

In [6]:
tmp_df = cleaned_df.copy()
nominal_val = cleaned_df.select_dtypes(include=['object']).columns
for index, values in tmp_df.iteritems():
    if index in nominal_val:
        unique = pd.unique(values)
        dic = []
        for i in values:
            dic.append(np.where(unique == i)[0][0])
        tmp_df[index] = dic

In [8]:
tmp_df.columns

Index(['Unnamed: 0', 'ARM', 'HR1_10SCALE', 'HR2_1', 'HR2_2', 'HR2_3', 'HR2_4',
       'HR2_5', 'HR3', 'HR4',
       ...
       'RUMORS1_USARMY_BROUGHT', 'RUMORS1_BAT_TO_HUMANS',
       'RUMORS1_BAT_ANIMAL_HUMANS', 'RUMORS1_WET_MARKET', 'RUMORS1_OTHER',
       'DEMOGRAPHIC1_EMPLOYED', 'DEMOGRAPHIC1_UNEMPLOYED',
       'DEMOGRAPHIC1_RETIRED', 'DEMOGRAPHIC1_STUDENTS', 'DEMOGRAPHIC1_OTHER'],
      dtype='object', length=216)

In [7]:
from sklearn.model_selection import train_test_split
X = tmp_df.drop("IDEOLOGY5",axis=1) #obamacare
y = tmp_df["IDEOLOGY5"]
#x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = .75, test_size = .25, random_state = 72)

### Feature selection with chi squared

In [10]:
from sklearn.feature_selection import SelectKBest, chi2
fs = SelectKBest(score_func=chi2, k='all')
fs.fit(x_train,y_train)
x_train_fs = fs.transform(x_train)
x_test_fs = fs.transform(x_test)

In [11]:
selected_features = []
for i in range(len(fs.scores_)):
    #print('Feature %s: %f' % (tmp_df.columns[i], fs.scores_[i]))
    if fs.scores_[i] > 100:
        print('Feature %s: %f' % (X.columns[i], fs.scores_[i]))
        selected_features.append(X.columns[i])
selected_features.remove("Unnamed: 0")
selected_features

Feature Unnamed: 0: 275.827388
Feature HR6: 101.234607
Feature MEDIA12: 101.260111
Feature IDEOLOGY6: 103.658266
Feature DEMOGRAPHIC4: 123.817113
Feature DEMOGRAPHIC5: 3800.760048


['HR6', 'MEDIA12', 'IDEOLOGY6', 'DEMOGRAPHIC4', 'DEMOGRAPHIC5']

Selected Features are 'HR6', 'MEDIA12', 'IDEOLOGY6', 'DEMOGRAPHIC4', 'DEMOGRAPHIC5' DEMOGRAPHIC 4 is nominal so it will have to be encoded.

### One Hot

In [12]:
selected_df = cleaned_df[selected_features].copy()
selected_df["IDEOLOGY5"] = cleaned_df["IDEOLOGY5"].copy()
selected_df

HR4  HR12_2  STIMULUS3  POLICY2 POLICY4  MEDIA6_1  MEDIA6_5  MEDIA12  \
0       2       0          0        2       A         1         3        1   
1       2       0          1        1       B         3         3        1   
2       0       0          0        0       B         1         1        0   
3       1       1          0        1       A         1         1        2   
4       4       1          1        1       A         1         3        1   
...   ...     ...        ...      ...     ...       ...       ...      ...   
1206    4       2          2        0       A         1         1        0   
1207    4       1          2        2       A         1         1        1   
1208    3       1          1        0       B         2         1        0   
1209    4       2          1        2       A         1         3        0   
1210    1       1          2        0       A         3         1        0   

     MEDIA17 CHILDCARE3  ... MEDIA16_SCHOOL MEDIA16_FOOD_STAMPS  \
0          B         No  ...              0                   1   
1          B         No  ...              0                   0   
2          B         No  ...              0                   0   
3          A         No  ...              1                   0   
4          B         No  ...              1                   1   
...      ...        ...  ...            ...                 ...   
1206       B         No  ...              0                   0   
1207       B         No  ...              0                   0   
1208       A         No  ...              0                   0   
1209       B         No  ...              0                   0   
1210       B         No  ...              0                   0   

     MEDIA16_COMMUNITY  MEDIA16_FOOD_ASSIS  RUMORS1_USARMY_BROUGHT  \
0                    0                   0                       0   
1                    0                   1                       0   
2                    0                   0                       0   
3                    0                   0                       0   
4                    0                   0                       0   
...                ...                 ...                     ...   
1206                 0                   0                       0   
1207                 0                   0                       0   
1208                 0                   1                       0   
1209                 0                   1                       0   
1210                 0                   1                       0   

     RUMORS1_BAT_ANIMAL_HUMANS DEMOGRAPHIC1_EMPLOYED DEMOGRAPHIC1_UNEMPLOYED  \
0                            0                     1                       0   
1                            1                     0                       0   
2                            0                     0                       0   
3                            0                     1                       0   
4                            0                     0                       0   
...                        ...                   ...                     ...   
1206                         0                     0                       0   
1207                         0                     0                       0   
1208                         0                     1                       0   
1209                         0                     1                       0   
1210                         0                     0                       0   

      DEMOGRAPHIC1_RETIRED  IDEOLOGY5  
0                        0          2  
1                        0          0  
2                        0          2  
3                        0          0  
4                        0          0  
...                    ...        ...  
1206                     0         -2  
1207                     0          0  
1208                     0          2  
1209                     0          0  
1210                  

In [13]:
nominal_df = selected_df[selected_df.select_dtypes(include=['object']).columns]
nominal_df

POLICY4 MEDIA17 CHILDCARE3 ELDERLYCARE1 IDEOLOGY1 IDEOLOGY3 DEMOGRAPHIC2  \
0          A       B         No           No         C         B          Yes   
1          B       B         No           No         B         B           No   
2          B       B         No           No         A         A           No   
3          A       A         No           No         C         D          Yes   
4          A       B         No           No         B         C          Yes   
...      ...     ...        ...          ...       ...       ...          ...   
1206       A       B         No          Yes         A         A          Yes   
1207       A       B         No           No         C         B          Yes   
1208       B       A         No           No         B         C          Yes   
1209       A       B         No           No         C         D          Yes   
1210       A       B         No          Yes         B         A           No   

     DEMOGRAPHIC6 DEMOGRAPHIC7  
0             Yes           No  
1             Yes           No  
2             Yes           No  
3             Yes           No  
4             Yes           No  
...           ...          ...  
1206          Yes           No  
1207          Yes          Yes  
1208          Yes           No  
1209          Yes           No  
1210          Yes           No  

[1211 rows x 9 columns]

In [14]:
nn_df = pd.DataFrame()
for i in nominal_df.columns:
    unique = pd.unique(selected_df[i].copy())
    count = 0
    for j in unique:
        name = i + "_" + str(j)
        tmp = []
        for k in nominal_df[i]:
            if k == j:
                tmp.append(1)
            else:
                tmp.append(0)
        count +=1
        nn_df[name] = tmp
nn_df

POLICY4_A  POLICY4_B  MEDIA17_B  MEDIA17_A  CHILDCARE3_No  \
0             1          0          1          0              1   
1             0          1          1          0              1   
2             0          1          1          0              1   
3             1          0          0          1              1   
4             1          0          1          0              1   
...         ...        ...        ...        ...            ...   
1206          1          0          1          0              1   
1207          1          0          1          0              1   
1208          0          1          0          1              1   
1209          1          0          1          0              1   
1210          1          0          1          0              1   

      CHILDCARE3_Yes  ELDERLYCARE1_No  ELDERLYCARE1_Yes  IDEOLOGY1_C  \
0                  0                1                 0            1   
1                  0                1                 0            0   
2                  0                1                 0            0   
3                  0                1                 0            1   
4                  0                1                 0            0   
...              ...              ...               ...          ...   
1206               0                0                 1            0   
1207               0                1                 0            1   
1208               0                1                 0            0   
1209               0                1                 0            1   
1210               0                0                 1            0   

      IDEOLOGY1_B  ...  IDEOLOGY3_B  IDEOLOGY3_A  IDEOLOGY3_D  IDEOLOGY3_C  \
0               0  ...            1            0            0            0   
1               1  ...            1            0            0            0   
2               0  ...            0            1            0            0   
3               0  ...            0            0            1            0   
4               1  ...            0            0            0            1   
...           ...  ...          ...          ...          ...          ...   
1206            0  ...            0            1            0            0   
1207            0  ...            1            0            0            0   
1208            1  ...            0            0            0            1   
1209            0  ...            0            0            1            0   
1210            1  ...            0            1            0            0   

      DEMOGRAPHIC2_Yes  DEMOGRAPHIC2_No  DEMOGRAPHIC6_Yes  DEMOGRAPHIC6_No  \
0                    1                0                 1                0   
1                    0                1                 1                0   
2                    0                1                 1                0   
3                    1                0                 1                0   
4                    1                0                 1                0   
...                ...              ...               ...              ...   
1206                 1                0                 1                0   
1207                 1                0                 1                0   
1208                 1                0                 1                0   
1209                 1                0                 1                0   
1210                 0                1                 1                0   

      DEMOGRAPHIC7_No  DEMOGRAPHIC7_Yes  
0                   1                 0  
1                   1                 0  
2                   1                 0  
3                   1                 0  
4                   1                 0  
...               ...               ...  
1206                1                 0  
1207                0                 1  
1208                1                 0  
1209                1                 0  


In [15]:
selected_df = selected_df.drop(nominal_df.columns,axis=1)
for index, value in nn_df.iteritems():
    selected_df[index] = value

### Now we have our selected features for our data. 

In [16]:
selected_df.to_csv("ideology5_encoded.csv")
selected_df

HR4  HR12_2  STIMULUS3  POLICY2  MEDIA6_1  MEDIA6_5  MEDIA12  IDEOLOGY6  \
0       2       0          0        2         1         3        1          2   
1       2       0          1        1         3         3        1          2   
2       0       0          0        0         1         1        0          0   
3       1       1          0        1         1         1        2          3   
4       4       1          1        1         1         3        1          2   
...   ...     ...        ...      ...       ...       ...      ...        ...   
1206    4       2          2        0         1         1        0          1   
1207    4       1          2        2         1         1        1          0   
1208    3       1          1        0         2         1        0          4   
1209    4       2          1        2         1         3        0          0   
1210    1       1          2        0         3         1        0          3   

      CULTURE1_6  DEMOGRAPHIC9  ...  IDEOLOGY3_B  IDEOLOGY3_A  IDEOLOGY3_D  \
0              3             3  ...            1            0            0   
1              2             3  ...            1            0            0   
2              5             0  ...            0            1            0   
3              4             2  ...            0            0            1   
4              3             2  ...            0            0            0   
...          ...           ...  ...          ...          ...          ...   
1206           5             0  ...            0            1            0   
1207           5             2  ...            1            0            0   
1208           4             1  ...            0            0            0   
1209           3             1  ...            0            0            1   
1210           2             4  ...            0            1            0   

      IDEOLOGY3_C  DEMOGRAPHIC2_Yes  DEMOGRAPHIC2_No  DEMOGRAPHIC6_Yes  \
0               0                 1                0                 1   
1               0                 0                1                 1   
2               0                 0                1                 1   
3               0                 1                0                 1   
4               1                 1                0                 1   
...           ...               ...              ...               ...   
1206            0                 1                0                 1   
1207            0                 1                0                 1   
1208            1                 1                0                 1   
1209            0                 1                0                 1   
1210            0                 0                1                 1   

      DEMOGRAPHIC6_No  DEMOGRAPHIC7_No  DEMOGRAPHIC7_Yes  
0                   0                1                 0  
1                   0                1                 0  
2                   0                1                 0  
3                   0                1                 0  
4                   0                1                 0  
...               ...              ...               ...  
1206                0                1                 0  
1207                0                0                 1  
1208                0                1                 0  
1209                0                1                 0  
1210                0                1                 0  

[1211 rows x 84 columns]